In [2]:
from ROCRL_environment import LinearSEMGenerator, LinearUtility, ROCRLEnvironment
#from algos import ROCRLLearner, ActionCovStats

In [3]:
n_latents = 5
sem = LinearSEMGenerator(n = n_latents, d = 10, seed = 0)
util = LinearUtility(n = n_latents, noise_std = 0.1, theta_dist = "rademacher", seed = 1)
env = ROCRLEnvironment(sem = sem, utility = util)

In [4]:
batch = env.step(num_samples = 4, kind = "none", return_latents = True)
print("X shape:", batch["X"].shape, "U.shape:", batch["U"].shape, "Z.shape:", batch["Z"].shape)

batch2 = env.step(num_samples = 4, action = {1,3}, kind = "soft")
print("mean utility observed vs intervened on: ", batch["U"].mean(), batch2["U"].mean())

X shape: (4, 10) U.shape: (4,) Z.shape: (4, 5)
mean utility observed vs intervened on:  1.0084824656743199 0.6600328994681209
